In [ ]:
import ast
import json
import pandas as pd

In [ ]:
train_data = pd.read_csv("train_data.csv").drop("Unnamed: 0", axis=1)

ParserError: ignored

In [ ]:
theme_to_paralist = train_data.groupby('Theme')['Paragraph'].apply(set).apply(list).to_frame().rename(columns={'Paragraph':'Paragraphs'})

In [ ]:
theme_to_paralist_dict = dict()
for i in range(theme_to_paralist.shape[0]): 
    theme_to_paralist_dict[theme_to_paralist.index.tolist()[i]] = theme_to_paralist.iloc[i].Paragraphs

In [ ]:
with open("task_1.json", "w") as outfile: json.dump(theme_to_paralist_dict, outfile)

In [ ]:
task_1 = train_data.drop(["Paragraph", "Answer_text", "Answer_start"], axis=1)

In [ ]:
from tqdm.notebook import tqdm
para_nos = []
for i in tqdm(range(train_data.shape[0])):
    list_of_paras = theme_to_paralist_dict[task_1.iloc[i].Theme]
    actual_para = train_data.iloc[i].Paragraph
    if(task_1.iloc[i].Answer_possible): para_nos.append([list_of_paras.index(actual_para)])
    else: para_nos.append([])
task_1['para_no'] = para_nos

  0%|          | 0/75055 [00:00<?, ?it/s]

In [ ]:
task_1[task_1.Theme!='Matter'].to_csv("task_1.csv")
train_data[train_data.Theme!='Matter'].to_csv("task_2.csv")

In [ ]:
task_1[task_1.Theme=='Matter'].to_csv("eval_task_1.csv")
train_data[train_data.Theme=='Matter'].to_csv("eval_task_2.csv")

##For Squad 2.0 BIDAF Model

In [13]:
task_2 = {"version" : "v2.0", "data" : []}

In [14]:
import pandas as pd
squad_train_data = pd.read_csv("eval_task_2.csv").drop("Unnamed: 0", axis=1)

In [15]:
from tqdm.notebook import tqdm

for i in tqdm(range(squad_train_data.shape[0])):

    # Get values
    curr_theme = squad_train_data.iloc[i].Theme
    curr_para = squad_train_data.iloc[i].Paragraph
    curr_q = squad_train_data.iloc[i].Question
    curr_possible = squad_train_data.iloc[i].Answer_possible
    curr_text = squad_train_data.iloc[i].Answer_text[2:-2]
    curr_start = squad_train_data.iloc[i].Answer_start[1:-1]
    
    if curr_possible == False:
      curr_question = {"question" : curr_q, "id" : str(i), "answers" : [], "is_impossible" : not curr_possible}
    else:
      curr_question = {"question" : curr_q, "id" : str(i), "answers" : [{"text" : curr_text, "answer_start" : curr_start}], "is_impossible" : not curr_possible}

    there = 0

    # If theme already present
    for themes in task_2["data"]:
      if "title" in themes and themes["title"] == curr_theme:
        there = 1
        para_there = 0
        for paras in themes["paragraphs"]:
          if "context" in paras and paras["context"] == curr_para:
            para_there = 1
            print("theme and para there --------------------------------------------")
            paras["qas"].append(curr_question)

        # Paragraph not there
        if para_there == 0:
          print("para not there --------------------------------------------")
          new_para_insert = {"qas": [curr_question], "context" : curr_para}
          themes["paragraphs"].append(new_para_insert)


    # If theme is not present
    if there == 0:
      new_theme_insert = {"title" : curr_theme, "paragraphs" : [{"qas" : [curr_question], "context" : curr_para}]}
      print("Theme not there ---------------------------------------------------")
      task_2["data"].append(new_theme_insert)
    



  0%|          | 0/269 [00:00<?, ?it/s]

Theme not there ---------------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
para not there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
theme and para there --------------------------------------------
para not there

In [ ]:
print(task_2)

{'version': 'v2.0', 'data': [{'title': 'Beyoncé', 'paragraphs': [{'qas': [{'question': "When did Beyonce leave Destiny's Child and become a solo singer?", 'id': '0', 'answers': [{'text': '2003', 'answer_start': '526'}], 'is_impossible': False}, {'question': 'What album made her a worldwide known artist?', 'id': '1', 'answers': [{'text': 'Dangerously in Love', 'answer_start': '505'}], 'is_impossible': False}], 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist 

In [16]:
import json
with open("eval_task_2.json", "w") as outfile: json.dump(task_2, outfile)